In [10]:
# Importando a biblioteca.
import pymysql

In [ ]:
# Conectando ao banco de dados por meio de uma função.
def connect():
    return pymysql.connect(
        host='localhost',
        user='root',
        password='password',
        database='projeto_sql',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

#### **Preenchendo a tabela breed_fur_color**

In [12]:
# Etapa 2: Conectar ao banco e preencher a tabela intermediária
try:
    conn = connect()
    print("✅ Conexão com o banco realizada com sucesso!")

    with conn.cursor() as cursor:

        # Etapa 3: Buscar os dados da tabela original
        cursor.execute("SELECT breed, fur_color FROM racas_caes")
        racas_dados = cursor.fetchall()

        for linha in racas_dados:
            breed_nome = linha['breed']
            cores = linha['fur_color'].split(', ')  # separa as cores da lista

            # Etapa 4: Obter o breed_id
            cursor.execute(
                "SELECT breed_id FROM breeds WHERE breed_name = %s", (breed_nome,))
            resultado_breed = cursor.fetchone()
            if not resultado_breed:
                print(
                    f"❌ Raça '{breed_nome}' não encontrada na tabela breeds.")
                continue

            breed_id = resultado_breed['breed_id']

            for cor in cores:
                # Etapa 5.1: Obter o fur_color_id
                cursor.execute(
                    "SELECT fur_color_id FROM fur_colors WHERE fur_color = %s", (cor,))
                resultado_cor = cursor.fetchone()
                if not resultado_cor:
                    print(
                        f"❌ Cor '{cor}' não encontrada na tabela fur_colors.")
                    continue

                fur_color_id = resultado_cor['fur_color_id']

                # Etapa 5.2: Verificar se já existe essa combinação
                cursor.execute("""
                    SELECT 1 FROM breed_fur_color
                    WHERE breed_id = %s AND fur_color_id = %s
                """, (breed_id, fur_color_id))

                existe = cursor.fetchone()

                if not existe:
                    cursor.execute("""
                        INSERT INTO breed_fur_color (breed_id, fur_color_id)
                        VALUES (%s, %s)
                    """, (breed_id, fur_color_id))

        conn.commit()
        print("\n✅ Preenchimento da tabela 'breed_fur_color' concluído com sucesso!")

except pymysql.MySQLError as e:
    print("❌ Erro ao acessar o banco de dados:")
    print(e)

finally:
    try:
        conn.close()
        print("🔒 Conexão encerrada com sucesso.")
    except:
        pass

✅ Conexão com o banco realizada com sucesso!

✅ Preenchimento da tabela 'breed_fur_color' concluído com sucesso!
🔒 Conexão encerrada com sucesso.


#### **Preenchendo a tabela breed_eye_color**

In [16]:
# Etapa 2: Preenchendo a tabela intermediária breed_eye_color
try:
    conn = connect()
    print("✅ Conexão com o banco realizada com sucesso!")

    with conn.cursor() as cursor:
        # Obter todas as raças e cores dos olhos da tabela bruta
        cursor.execute("SELECT breed, eyes_color FROM racas_caes;")
        dados = cursor.fetchall()

        for linha in dados:
            breed_name = linha['breed']
            eyes_color = linha['eyes_color']

            # Ignorar valores vazios ou nulos
            if not eyes_color:
                continue

            # Buscar o ID da raça
            cursor.execute(
                "SELECT breed_id FROM breeds WHERE breed_name = %s;", (breed_name,))
            resultado_raca = cursor.fetchone()
            if not resultado_raca:
                print(
                    f"❌ Raça '{breed_name}' não encontrada na tabela breeds.")
                continue
            breed_id = resultado_raca['breed_id']

            # Separar múltiplas cores de olho
            cores = [cor.strip() for cor in eyes_color.split(',')]

            for cor in cores:
                # Buscar o ID da cor de olho
                cursor.execute(
                    "SELECT eye_color_id FROM eye_colors WHERE eye_color = %s;", (cor,))
                resultado_cor = cursor.fetchone()
                if not resultado_cor:
                    print(
                        f"❌ Cor de olho '{cor}' não encontrada na tabela eye_colors.")
                    continue
                eye_color_id = resultado_cor['eye_color_id']

                # Inserir na tabela intermediária (evitar duplicatas)
                try:
                    cursor.execute(
                        "INSERT IGNORE INTO breed_eye_color (breed_id, eye_color_id) VALUES (%s, %s);",
                        (breed_id, eye_color_id)
                    )
                except Exception as e:
                    print(
                        f"⚠️ Erro ao inserir ({breed_id}, {eye_color_id}): {e}")

        conn.commit()
        print("\n✅ Preenchimento da tabela 'breed_eye_color' concluído com sucesso!")

except pymysql.MySQLError as e:
    print("❌ Erro ao acessar o banco de dados:")
    print(e)

finally:
    try:
        conn.close()
        print("🔒 Conexão encerrada com sucesso.")
    except:
        pass

✅ Conexão com o banco realizada com sucesso!

✅ Preenchimento da tabela 'breed_eye_color' concluído com sucesso!
🔒 Conexão encerrada com sucesso.


#### **Preenchendo a tabela breed_hp**

In [23]:
try:
    conn = connect()
    print("✅ Conexão com o banco realizada com sucesso!")

    with conn.cursor() as cursor:
        # Buscar todas as raças e seus problemas de saúde
        cursor.execute(
            "SELECT breed, health_problems FROM racas_caes WHERE health_problems IS NOT NULL AND health_problems != '';")
        registros = cursor.fetchall()

        for linha in registros:
            breed_name = linha["breed"].strip()
            problemas = [p.strip() for p in linha["health_problems"].split(",")]

            # Buscar breed_id
            cursor.execute(
                "SELECT breed_id FROM breeds WHERE breed_name = %s;", (breed_name,))
            resultado_breed = cursor.fetchone()
            if not resultado_breed:
                print(
                    f"❌ Raça '{breed_name}' não encontrada na tabela breeds.")
                continue
            breed_id = resultado_breed["breed_id"]

            for problema in problemas:
                # Buscar problem_id
                cursor.execute(
                    "SELECT problem_id FROM health_problems WHERE problem = %s;", (problema,))
                resultado_prob = cursor.fetchone()
                if not resultado_prob:
                    print(
                        f"❌ Problema de saúde '{problema}' não encontrado na tabela health_problems.")
                    continue
                problem_id = resultado_prob["problem_id"]

                # Verificar se já existe antes de inserir
                cursor.execute(
                    "SELECT 1 FROM breed_hp WHERE breed_id = %s AND problem_id = %s;",
                    (breed_id, problem_id)
                )
                if cursor.fetchone():
                    continue  # já existe, não insere novamente

                # Inserir na tabela intermediária
                cursor.execute(
                    "INSERT INTO breed_hp (breed_id, problem_id) VALUES (%s, %s);",
                    (breed_id, problem_id)
                )

        conn.commit()
        print("✅ Preenchimento da tabela 'breed_hp' concluído com sucesso!")

except pymysql.MySQLError as e:
    print("❌ Erro ao acessar o banco de dados:")
    print(e)

finally:
    try:
        conn.close()
        print("🔒 Conexão encerrada com sucesso.")
    except:
        pass

✅ Conexão com o banco realizada com sucesso!
✅ Preenchimento da tabela 'breed_hp' concluído com sucesso!
🔒 Conexão encerrada com sucesso.


### **Preenchendo a tabela breed_traits**

In [24]:
try:
    conn = connect()
    print("✅ Conexão com o banco realizada com sucesso!")

    with conn.cursor() as cursor:
        # Buscar raças com seus traços de personalidade
        cursor.execute(
            "SELECT breed, character_traits FROM racas_caes WHERE character_traits IS NOT NULL AND character_traits != '';")
        registros = cursor.fetchall()

        for linha in registros:
            breed_name = linha["breed"].strip()
            traits = [t.strip() for t in linha["character_traits"].split(",")]

            # Buscar o breed_id
            cursor.execute(
                "SELECT breed_id FROM breeds WHERE breed_name = %s;", (breed_name,))
            resultado_breed = cursor.fetchone()
            if not resultado_breed:
                print(
                    f"❌ Raça '{breed_name}' não encontrada na tabela breeds.")
                continue
            breed_id = resultado_breed["breed_id"]

            for trait in traits:
                # Buscar trait_id
                cursor.execute(
                    "SELECT trait_id FROM character_traits WHERE trait = %s;", (trait,))
                resultado_trait = cursor.fetchone()
                if not resultado_trait:
                    print(
                        f"❌ Traço de personalidade '{trait}' não encontrado na tabela character_traits.")
                    continue
                trait_id = resultado_trait["trait_id"]

                # Verificar se já existe antes de inserir
                cursor.execute(
                    "SELECT 1 FROM breed_traits WHERE breed_id = %s AND trait_id = %s;",
                    (breed_id, trait_id)
                )
                if cursor.fetchone():
                    continue  # já existe

                # Inserir na tabela intermediária
                cursor.execute(
                    "INSERT INTO breed_traits (breed_id, trait_id) VALUES (%s, %s);",
                    (breed_id, trait_id)
                )

        conn.commit()
        print("✅ Preenchimento da tabela 'breed_traits' concluído com sucesso!")

except pymysql.MySQLError as e:
    print("❌ Erro ao acessar o banco de dados:")
    print(e)

finally:
    try:
        conn.close()
        print("🔒 Conexão encerrada com sucesso.")
    except:
        pass

✅ Conexão com o banco realizada com sucesso!
✅ Preenchimento da tabela 'breed_traits' concluído com sucesso!
🔒 Conexão encerrada com sucesso.
